In [44]:
import pandas as pd
import numpy as np 
import requests
import json
from pprint import pprint
from config import key_dict
from datetime import datetime
import time
import gmaps

In [45]:
from ipywidgets.embed import embed_minimal_html
import ipywidgets as widgets
from IPython.display import display

gmaps.configure(api_key = key_dict['google'])

In [84]:
test_final_df = pd.read_csv('Resources/aqi_data.csv')
test_final_df.Date = pd.to_datetime(test_final_df.Date)
test_final_df = test_final_df.set_index('Date')
test_final_df['Lat'] = 32.7157
test_final_df['Long'] = -117.1611
test_final_df.rename(columns = {"Overall AQI Value":"AQI"}, inplace = True)
test_final_df['AQI'] = test_final_df['AQI'].astype(float)
test_final_df = test_final_df.replace(".",None)
test_final_df.iloc[0,8] = None
test_final_df[['CO','Ozone','SO2','PM10','PM25', 'NO2']] = \
    test_final_df[['CO','Ozone','SO2','PM10','PM25', 'NO2']].astype(float)
#final data set up for the whole data gathered

In [47]:

# credit to pbugnion on github; small adjustments for personal keys/indexes, accepting dataframe format, etc
# https://github.com/pbugnion/gmaps/issues/216


class HeatmapAnimation(object):
    
    def __init__(self, datasets):
        self._datasets = datasets
        self._figure = gmaps.figure(center=(32.75, -117.0), zoom_level=10)
        self._index_count = 0
        self._current_index = datasets.index.unique()[self._index_count]
        self._heatmap = gmaps.heatmap_layer(locations = datasets.loc[datasets.index == self._current_index,['Lat','Long']],
                                            weights = datasets.loc[datasets.index == self._current_index,'AQI'],
                                           max_intensity = datasets['AQI'].max(), point_radius = 40)
        self._figure.add_layer(self._heatmap)
        self._out = widgets.HTML()
        self._out.value = str(self._current_index.date())
        
    def render(self):
        return display(self._figure),display(self._out)
    
    def start_animation(self):
        while True:
            self._index_count = self._index_count + 1
            try:
                self._current_index = self._datasets.index.unique()[self._index_count]
            except IndexError:
                self._index_count = 0
                self._current_index = self._datasets.index.unique()[self._index_count]
            self._render_current_dataset()
            time.sleep(0.1)
    
    def _render_current_dataset(self):
        self._heatmap.locations = self._datasets.loc[self._datasets.index == 
                                               self._current_index,['Lat','Long']] # update the locations drawn on the heatmap
        self._heatmap.weights = self._datasets.loc[self._datasets.index == 
                                             self._current_index,'AQI']
        self._out.value = str(self._current_index.date())


In [85]:
reduced_df = test_final_df.drop(index = test_final_df.loc[test_final_df.index.month.isin([1,2,7,8,9,10,11,12]),
                                             ].index)

#reduce to march-april-may

In [125]:
animation = HeatmapAnimation(reduced_df)
animation.render()
animation.start_animation()


Figure(layout=FigureLayout(height='420px'))

HTML(value='2018-03-01')

KeyboardInterrupt: 

In [124]:

reduced_df.groupby([reduced_df.index.year, reduced_df.index.month]).mean().round(3) #.loc[2018,3]

AQI     CO   Ozone    SO2    PM10    PM25     NO2     Lat  \
Date Date                                                                 
2018 3     53.161  7.581  51.516  0.000  16.484  38.452  26.742  32.716   
     4     73.133  7.233  69.867  0.000  23.267  59.233  23.400  32.716   
     5     68.774  5.742  67.677  0.000  21.129  46.806  19.968  32.716   
     6     90.400  5.767  88.867  0.133  30.200  55.267  17.733  32.716   
2019 3     53.581  6.613  50.129  0.000  18.194  43.645  23.935  32.716   
     4     68.167  6.300  61.300  0.000  19.767  56.767  22.433  32.716   
     5     55.645  5.194  51.742  0.000  15.419  49.129  16.677  32.716   
     6     65.767  5.767  61.667  0.000  15.967  55.067  15.400  32.716   
2020 3     46.129  6.065  41.806  0.000  15.613  36.710  20.484  32.716   
     4     60.667  5.100  54.300  0.000  21.800  50.833  18.267  32.716   
     5     70.871  5.645  61.710  0.032  23.387  62.226  15.129  32.716   
     6     68.567  5.967  60.800  0.000  26.133  58.733  17.167  32.716   
2021 3     47.375    NaN  47.083    NaN     NaN  32.417     NaN  32.716   

              Long  
Date Date           
2018 3    -117.161  
     4    -117.161  
     5    -117.161  
     6    -117.161  
2019 3    -117.161  
     4    -117.161  
     5    -117.161  
     6    -117.161  
2020 3    -117.161  
     4    -117.161  
     5    -117.161  
     6    -117.161  
2021 3    -117.161